# Financial Analysis 
## (using multi-agent collaboration system)

In this project, we will use 4 agents

- Data Analyst Agent 
- Trading Strategy Agent
- Execution Agent
- Risk Management Agent



Using the 4 agents, we work towards 4 tasks

- Data Analysis Task (for Data Analyst)
- Strategy Development Task (for Trading Strategy agent)
- Execution Planning Task (for Execution Agent)
-  Risk Assessment Task (for Risk Management Agent)

In this project, we will use a **Hierarchial Process**, where crewAI creates a manager to delegate task to all the 4 agents

## Step 01. Installing crewai and crewai_tools libraries

**Note:** crewai requires Python version >=3.10 and <=3.13

In [9]:
import sys
print(sys.version)

3.10.14 | packaged by conda-forge | (main, Mar 20 2024, 12:45:18) [GCC 12.3.0]


In [9]:
import sys
print(sys.version)

3.10.14 | packaged by conda-forge | (main, Mar 20 2024, 12:45:18) [GCC 12.3.0]


In [11]:
# !pip install crewai_tools

In [12]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [9]:
import sys
print(sys.version)

3.10.14 | packaged by conda-forge | (main, Mar 20 2024, 12:45:18) [GCC 12.3.0]


## Step 02. Importing modules, LLMs, tools

**NOTE:** To create Agent class objects, we need to specify OPEN_AI model name and api key as environment variables

**NOTE:** To run crewai's SerperDevTool, we need SERPER_API_KEY as environment variables

In [13]:
import boto3
from langchain_aws import ChatBedrock

# Set up Amazon Bedrock Boto3 client
bedrock_runtime = boto3.client(service_name="bedrock-runtime", region_name="us-east-1")

# Configure the model to use
model_id = "anthropic.claude-3-haiku-20240307-v1:0"
model_kwargs = {
    "max_tokens": 2048,
    "temperature": 0.1,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman"],
}

# Create the LangChain Chat object
llm = ChatBedrock(
    client=bedrock_runtime,
    model_id=model_id,
    model_kwargs=model_kwargs,
)

In [14]:
from crewai import Crew, Agent, Task

import os
# os.environ['OPENAI_MODEL_NAME'] = "gpt-4-turbo"
# os.environ['OPENAI_API_KEY'] = "Your OPEN AI Api Key"
os.environ['SERPER_API_KEY'] = "0c13a642c977cf1ffff2f262cda8ddce9c71d590"

### crewAI Tools

In [15]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Step 03. Creating Agents

### Agent 1: Data Analyst Agent

In [16]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    llm=llm,
    allow_delegation=True,
    verbose=True,
    tools=[search_tool, scrape_tool]
)

### Agent 2: Trading Strategy Agent

In [17]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    llm=llm,
    allow_delegation=True,
    tools=[search_tool, scrape_tool],
    verbose=True
)

### Agent 3: Execution Agent

In [18]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    llm=llm,
    allow_delegation=True,
    tools=[search_tool, scrape_tool],
    verbose=True
)

### Agent 4: Risk Management Agent

In [19]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    llm=llm,
    allow_delegation=True,
    tools=[search_tool, scrape_tool],
    verbose=True
)

## Step 04. Creating Tasks

In [20]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [21]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [22]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [23]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Step 06. Creating Crew

### Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [24]:
from crewai import Crew, Process
# from langchain_openai import ChatOpenAI

In [25]:
# define the crew with agent and tasks
financial_trading_crew = Crew (
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    verbose=True,
    process=Process.hierarchical,
    manager_llm=llm #ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)
)

### Running the crew

In [26]:
# specify the input for crew
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [27]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [2024-07-25 01:19:49][DEBUG]: == Working Agent: Crew Manager
 [2024-07-25 01:19:49][INFO]: == Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
Thought: To complete this task, I will need to gather more information from the Data Analyst. I will delegate the task of continuously monitoring and analyzing market data for AAPL, and ask relevant questions to get the necessary insights.

Action: Delegate work to coworker
Action Input: {
  "task": "Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.",
  "context": "We need to provide insights and alerts about significant market opportunities or threats for AAPL. I have the overall goal of managing the team to complete the task in the best way possi

- Display the final result as Markdown.

In [29]:
from IPython.display import Markdown
result_str = str(result)  # Convert result to string
Markdown(result_str)  # Display as Markdown

Comprehensive Risk Analysis Report for AAPL Trading Strategies:

Potential Risks:

1. Regulatory Risks:
   - Apple faces potential regulatory challenges, particularly around its App Store and search default agreements, which generate significant revenue. Unfavorable regulatory decisions could impact Apple's profitability and stock performance.
   - Mitigation: Closely monitor regulatory developments and assess their potential impact on Apple's business model. Diversify revenue streams to reduce reliance on any single revenue source.

2. Competitive Threats:
   - Apple operates in a highly competitive technology industry, facing challenges from rivals in smartphones, tablets, laptops, and other product categories. Increased competition could erode Apple's market share and pricing power.
   - Mitigation: Analyze industry trends, monitor competitor activities, and ensure Apple maintains its innovative edge and product differentiation.

3. Valuation Risks:
   - Apple's stock is currently trading at a premium valuation, which makes it more susceptible to market corrections and volatility. A significant drop in the stock price could lead to substantial losses for traders.
   - Mitigation: Employ risk management strategies, such as stop-loss orders and position sizing, to limit potential losses. Diversify your trading portfolio to reduce overall exposure to AAPL.

4. Headline Risks:
   - Apple is a high-profile company, and any negative news or events, such as legal disputes, product recalls, or executive changes, can significantly impact the stock price in the short term.
   - Mitigation: Stay informed about the latest developments and news surrounding Apple. Adjust trading strategies and risk management practices accordingly to navigate potential volatility.

5. Macroeconomic Risks:
   - Apple's performance is influenced by broader economic conditions, such as consumer spending, inflation, and interest rate changes. Adverse macroeconomic factors could affect Apple's sales and profitability.
   - Mitigation: Monitor economic indicators and adjust trading strategies based on the prevailing market environment. Consider diversifying your portfolio to include other sectors and asset classes to mitigate overall market risk.

To mitigate these risks, it is crucial to develop a comprehensive trading plan that aligns with your risk tolerance and investment objectives. This plan should include:

- Thorough research and analysis of Apple's fundamentals, industry trends, and market dynamics.
- Disciplined risk management practices, such as position sizing, stop-loss orders, and diversification.
- Continuous monitoring and adaptation of trading strategies based on changing market conditions.
- Maintaining emotional control and discipline, avoiding impulsive decision-making.
- Ongoing education and skill development to enhance your trading expertise.

By carefully considering these risks and implementing appropriate risk management strategies, you can navigate the trading of AAPL more effectively and align your trading activities with the firm's risk tolerance.